Disciplina: Estrutura de Dados Avançados

Período: 2025.1

Professor: Victor Almeida Campos

Aluno: Davi Teixeira Silva

# Estrutura van Emde Boas tree (VEB tree)

Descrição geral do trabalho:

O objetivo do trabalho é implementar uma estrutura van Emde Boas com espaço linear no número de elementos, ou seja, usando tabela de dispersão com table doubling/halving. A estrutura deve ser feita para o tamanho de palavra de 32 bits. A entrada do programa é um arquivo de texto com várias operações sobre a estrutura (uma por linha). A saída do programa será um arquivo de texto com as operações contidas no arquivo de entrada e seus resultados em seguida, caso existam.

## Operações

### Inclusão:

Uma operação de inclusão será identificada por uma linha escrito INC seguido de um espaço e depois um inteiro. Este elemento deve ser incluído na estrutura de dados.
- Exemplo na entrada:

```
INC 13
No arquivo de saída será gerado:
INC 13
```

### Remoção:

Uma operação de remoção será identificada por uma linha escrito REM seguido de um espaço e depois um inteiro. Este valor deve ser removido da estrutura, se existir.
- Exemplo na entrada:

```
REM 42
No arquivo de saída será gerado:REM 42
```

### Sucessor:

Uma operação de sucessor será identificada por uma linha escrito SUC seguido de um espaço e depois um inteiro. O sucessor do número x é o menor valor na estrutura que é estritamente maior que x. Esta operação gera como resultado o sucessor do inteiro dado como entrada.
- Exemplo na entrada:

```
SUC 50
No arquivo de saída será gerado:
SUC 50
80
```

### Predecessor:

Uma operação de predecessor será identificada por uma linha escrito PRE seguido de um espaço e depois um inteiro. O predecessor do número x é o maior valor na estrutura que é estritamente menor que x. Esta operação gera como resultado o predecessor do inteiro dado como entrada.
- Exemplo na entrada:

```
PRE 50
No arquivo de saída será gerado:
PRE 50
21
```

### Imprimir:

Uma operação de impressão será identificada por uma linha escrito IMP. A impressão de uma van Emde Boas oferece uma visão a partir do primeiro nível da estrutura. Neste caso, você deve imprimir o valor mínimo da estrutura, os valores que estão nos clusters não vazios.
- Exemplo na entrada:

```
IMP
No arquivo de saída será gerado:
IMP
Min: 102, C[0]: 268, 322, 14756, C[3]: 456, 728, 152, C[65535]: 0, 65535
```

# Introdução

A estrutura van Emde Boas (vEB) é uma árvore recursiva que permite realizar operações de dicionário como inserção, remoção, busca de sucessor e antecessor em tempo `O(log log U)`, onde `U` é o universo de chaves. Essa performance extremamente eficiente é obtida por meio de uma divisão recursiva do universo em blocos menores, otimizando o acesso a elementos.

In [34]:
# --------------------------------------------------
# 1. IMPORTAÇÃO DE BIBLIOTECAS
# --------------------------------------------------
import math
from typing import Optional, List, Dict, Set

# --------------------------------------------------
# 2. DEFINIÇÃO DA CLASSE HASH TABLE
# Tabela de espalhamento com ajuste dinâmico (table doubling/halving)
# --------------------------------------------------
class HashTable:

  # --------------------------------------------------
  # 2.1. MÉTODO CONSTRUTOR
  # Inicializa a tabela com capacidade inicial de 8 e nenhuma chave armazenada
  # --------------------------------------------------
  def __init__(self):
    self.capacidade = 8
    self.tamanho = 0
    self.dados = [None] * self.capacidade
    self.removidos = [False] * self.capacidade

  # --------------------------------------------------
  # 2.2. FUNÇÃO DE DISPERSÃO (HASHING)
  # Calcula o índice de espalhamento de uma chave
  # --------------------------------------------------
  def _espalhar(self, chave: int) -> int:
    return chave % self.capacidade

  # --------------------------------------------------
  # 2.3. AJUSTE DE TAMANHO DA TABELA
  # Realiza o redimensionamento da tabela (dobrando ou reduzindo)
  # --------------------------------------------------
  def _ajustar_tamanho(self, nova_capacidade: int):
    dados_antigos = self.dados
    removidos_antigos = self.removidos
    capacidade_antiga = self.capacidade

    self.capacidade = nova_capacidade
    self.dados = [None] * self.capacidade
    self.removidos = [False] * self.capacidade
    tamanho_antigo = self.tamanho
    self.tamanho = 0

    for i in range(capacidade_antiga):
      if dados_antigos[i] is not None and not removidos_antigos[i]:
        self.adicionar(dados_antigos[i])

  # --------------------------------------------------
  # 2.4. VERIFICA SE HÁ NECESSIDADE DE REDIMENSIONAR
  # Com base no fator de carga atual
  # --------------------------------------------------
  def _verificar_ajuste(self) -> bool:
    fator_de_carga = self.tamanho / self.capacidade
    return fator_de_carga > 0.7 or (self.capacidade > 8 and fator_de_carga < 0.2)

  # --------------------------------------------------
  # 2.5. INSERÇÃO DE CHAVE
  # Adiciona uma nova chave, redimensionando se necessário
  # --------------------------------------------------
  def adicionar(self, chave: int) -> bool:
    if self.existe(chave):
      return False

    if self._verificar_ajuste():
      if self.tamanho / self.capacidade > 0.7:
        self._ajustar_tamanho(self.capacidade * 2)
      else:
        self._ajustar_tamanho(max(8, self.capacidade // 2))

    indice = self._espalhar(chave)
    while self.dados[indice] is not None and not self.removidos[indice]:
      indice = (indice + 1) % self.capacidade

    self.dados[indice] = chave
    self.removidos[indice] = False
    self.tamanho += 1
    return True

  # --------------------------------------------------
  # 2.6. REMOÇÃO DE CHAVE
  # Marca o item como removido e ajusta tamanho se necessário
  # --------------------------------------------------
  def excluir(self, chave: int) -> bool:
    indice = self._espalhar(chave)

    while self.dados[indice] is not None:
      if self.dados[indice] == chave and not self.removidos[indice]:
        self.removidos[indice] = True
        self.tamanho -= 1

        if self._verificar_ajuste():
          self._ajustar_tamanho(max(8, self.capacidade // 2))

        return True
      indice = (indice + 1) % self.capacidade

    return False

  # --------------------------------------------------
  # 2.7. CONSULTA DE EXISTÊNCIA DE CHAVE
  # Retorna True se a chave estiver na tabela
  # --------------------------------------------------
  def existe(self, chave: int) -> bool:
    indice = self._espalhar(chave)

    while self.dados[indice] is not None:
      if self.dados[indice] == chave and not self.removidos[indice]:
        return True
      indice = (indice + 1) % self.capacidade

    return False

  # --------------------------------------------------
  # 2.8. RETORNO DAS CHAVES ATIVAS
  # Retorna uma lista ordenada com as chaves ainda ativas
  # --------------------------------------------------
  def obter_chaves(self) -> List[int]:
    chaves = []
    for i in range(self.capacidade):
      if self.dados[i] is not None and not self.removidos[i]:
        chaves.append(self.dados[i])
    return sorted(chaves)

  # --------------------------------------------------
  # 2.9. VERIFICA SE A TABELA ESTÁ VAZIA
  # --------------------------------------------------
  def esta_vazia(self) -> bool:
    return self.tamanho == 0

# --------------------------------------------------
# 3. ESTRUTURA VAN EMDE BOAS (VEB TREE)
# --------------------------------------------------
# Implementação otimizada em espaço da estrutura van Emde Boas,
# permitindo operações como inserção, remoção, busca, sucessor e antecessor
# em tempo quase logarítmico com respeito ao tamanho do universo.
# --------------------------------------------------
class EstruturaVEB:

  # --------------------------------------------------
  # 3.1. CONSTRUTOR DA ESTRUTURA
  # --------------------------------------------------
  def __init__(self, tamanho_universo: int):
    self.universo = tamanho_universo
    self.menor = None
    self.maior = None

    if tamanho_universo <= 2:
      self.resumo = None
      self.clusters = None
    else:
      self.tam_cluster = int(math.sqrt(tamanho_universo))
      self.total_clusters = int(math.ceil(tamanho_universo / self.tam_cluster))

      self.clusters = HashTable()
      self.objetos_cluster = {}
      self.resumo = EstruturaVEB(self.total_clusters)

  # --------------------------------------------------
  # 3.2. FUNÇÕES AUXILIARES DE POSICIONAMENTO
  # --------------------------------------------------
  def _parte_alta(self, x: int) -> int:
    # Parte alta do índice x (cluster)
    return x // self.tam_cluster

  def _parte_baixa(self, x: int) -> int:
    # Parte baixa do índice x (posição dentro do cluster)
    return x % self.tam_cluster

  def _combinar(self, alta: int, baixa: int) -> int:
    # Combina parte alta e parte baixa em índice do universo
    return alta * self.tam_cluster + baixa

  def _obter_cluster(self, alta: int) -> 'EstruturaVEB':
    # Retorna o cluster correspondente, criando-o se necessário
    if alta not in self.objetos_cluster:
      self.objetos_cluster[alta] = EstruturaVEB(self.tam_cluster)
      self.clusters.adicionar(alta)
    return self.objetos_cluster[alta]

  def _remover_cluster_se_vazio(self, alta: int):
    # Remove cluster da estrutura se estiver vazio
    if alta in self.objetos_cluster:
      cluster = self.objetos_cluster[alta]
      if cluster.menor is None:
        del self.objetos_cluster[alta]
        self.clusters.excluir(alta)

  # --------------------------------------------------
  # 3.3. INSERÇÃO DE ELEMENTOS
  # --------------------------------------------------
  def adicionar(self, x: int) -> bool:
    if x < 0 or x >= self.universo:
      return False

    if self.menor is None:
      self.menor = self.maior = x
      return True

    if x == self.menor or x == self.maior:
      return False

    if x < self.menor:
      x, self.menor = self.menor, x

    if x > self.maior:
      self.maior = x

    if self.universo <= 2:
      return True

    alta = self._parte_alta(x)
    baixa = self._parte_baixa(x)

    cluster = self._obter_cluster(alta)

    if cluster.menor is None:
      self.resumo.adicionar(alta)

    cluster.adicionar(baixa)
    return True

  # --------------------------------------------------
  # 3.4. REMOÇÃO DE ELEMENTOS
  # --------------------------------------------------
  def excluir(self, x: int) -> bool:
    if x < 0 or x >= self.universo or self.menor is None:
      return False

    # Caso trivial: apenas um elemento
    if self.menor == self.maior:
      if x == self.menor:
        self.menor = self.maior = None
        return True
      return False

    if self.universo <= 2:
      if x == self.menor:
        self.menor = self.maior
        return True
      elif x == self.maior:
        self.maior = self.menor
        return True
      return False

    # Atualiza menor, se necessário
    if x == self.menor:
      primeiro_cluster = self.resumo.menor
      if primeiro_cluster is None:
        self.menor = self.maior
        return True

      cluster_obj = self.objetos_cluster[primeiro_cluster]
      x = self._combinar(primeiro_cluster, cluster_obj.menor)
      self.menor = x

      alta = self._parte_alta(x)
      baixa = self._parte_baixa(x)

      if alta not in self.objetos_cluster:
        return False

      cluster = self.objetos_cluster[alta]
      removido = cluster.excluir(baixa)

      if not removido:
        return False

      if cluster.menor is None:
        self.resumo.excluir(alta)
        self._remover_cluster_se_vazio(alta)

      if x == self.maior:
        max_resumo = self.resumo.maior
        if max_resumo is None:
          self.maior = self.menor
        else:
          cluster_max = self.objetos_cluster[max_resumo]
          self.maior = self._combinar(max_resumo, cluster_max.maior)

    # Remoção regular
    alta = self._parte_alta(x)
    baixa = self._parte_baixa(x)

    if alta not in self.objetos_cluster:
      return False

    cluster = self.objetos_cluster[alta]
    removido = cluster.excluir(baixa)

    if not removido:
      return False

    if cluster.menor is None:
      self.resumo.excluir(alta)
      self._remover_cluster_se_vazio(alta)

    # Ajustar o maior se necessário
    if x == self.maior:
      if self.resumo.maior is None:
        self.maior = self.menor
      else:
        cluster_max = self.objetos_cluster[self.resumo.maior]
        self.maior = self._combinar(self.resumo.maior, cluster_max.maior)

    return True

  # --------------------------------------------------
  # 3.5. VERIFICA EXISTÊNCIA DE UM ELEMENTO
  # --------------------------------------------------
  def existe(self, x: int) -> bool:
    if x < 0 or x >= self.universo or self.menor is None:
      return False

    if x == self.menor or x == self.maior:
      return True

    if self.universo <= 2:
      return False

    alta = self._parte_alta(x)
    baixa = self._parte_baixa(x)

    if alta not in self.objetos_cluster:
      return False

    return self.objetos_cluster[alta].existe(baixa)

  # --------------------------------------------------
  # 3.6. ENCONTRA O SUCESSOR DE UM ELEMENTO
  # --------------------------------------------------
  def sucessor_de(self, x: int) -> Optional[int]:
    if self.universo <= 2:
      if x == 0 and self.maior == 1:
        return 1
      return None

    if self.menor is not None and x < self.menor:
      return self.menor

    alta = self._parte_alta(x)
    baixa = self._parte_baixa(x)

    if alta in self.objetos_cluster:
      cluster = self.objetos_cluster[alta]
      if cluster.maior is not None and baixa < cluster.maior:
        s_baixa = cluster.sucessor_de(baixa)
        if s_baixa is not None:
          return self._combinar(alta, s_baixa)

    proximo_cluster = self.resumo.sucessor_de(alta)
    if proximo_cluster is None:
      return None

    if proximo_cluster in self.objetos_cluster:
      cluster = self.objetos_cluster[proximo_cluster]
      if cluster.menor is not None:
        return self._combinar(proximo_cluster, cluster.menor)

    return None

  # --------------------------------------------------
  # 3.7. ENCONTRA O ANTECESSOR DE UM ELEMENTO
  # --------------------------------------------------
  def antecessor_de(self, x: int) -> Optional[int]:
    if self.universo <= 2:
      if x == 1 and self.menor == 0:
        return 0
      return None

    if self.maior is not None and x > self.maior:
      return self.maior

    alta = self._parte_alta(x)
    baixa = self._parte_baixa(x)

    if alta in self.objetos_cluster:
      cluster = self.objetos_cluster[alta]
      if cluster.menor is not None and baixa > cluster.menor:
        p_baixa = cluster.antecessor_de(baixa)
        if p_baixa is not None:
          return self._combinar(alta, p_baixa)

    anterior_cluster = self.resumo.antecessor_de(alta)
    if anterior_cluster is None:
      if self.menor is not None and x > self.menor:
        return self.menor
      return None

    if anterior_cluster in self.objetos_cluster:
      cluster = self.objetos_cluster[anterior_cluster]
      if cluster.maior is not None:
        return self._combinar(anterior_cluster, cluster.maior)

    # menor seja retornado se for valido
    if self.menor is not None and x > self.menor:
      return self.menor

    return None

  # --------------------------------------------------
  # 3.8. LISTA ELEMENTOS AGRUPADOS POR CLUSTERS
  # --------------------------------------------------
  def listar_por_cluster(self) -> Dict[int, List[int]]:
    resultado = {}

    if self.menor is not None:
      resultado['min'] = self.menor

    if self.universo <= 2:
      return resultado

    for idx in sorted(self.clusters.obter_chaves()):
      if idx in self.objetos_cluster:
        cluster = self.objetos_cluster[idx]
        elementos = []

        if cluster.menor is not None:
          elementos.append(self._combinar(idx, cluster.menor))

          if cluster.maior != cluster.menor:
            for i in range(cluster.universo):
              if i != cluster.menor and cluster.existe(i):
                elementos.append(self._combinar(idx, i))

            if cluster.maior is not None:
              elementos.append(self._combinar(idx, cluster.maior))

        if elementos:
          resultado[idx] = sorted(list(set(elementos)))

    return resultado


# --------------------------------------------------
# 4. TESTES DE VALIDAÇÃO DAS OPERAÇÕES
# --------------------------------------------------
def testar_veb():

  """
  Executa uma sequência de testes na estrutura EstruturaVEB para validar
  operações básicas como inserção, busca, sucessor, antecessor e exclusão.
  """

  print("=== Iniciando testes na estrutura EstruturaVEB ===")

  # Criando a estrutura para universo de tamanho 16
  veb = EstruturaVEB(16)

  # 1. Teste de inserção
  print("\n[1] Inserção de elementos: 1, 6, 8, 10, 12")
  for valor in [1, 6, 8, 10, 12]:
    veb.adicionar(valor)
    print(f" -> Inserido: {valor}")

  # 2. Teste de verificação de existência
  print("\n[2] Verificação de existência:")
  for valor in [0, 1, 5, 6, 15]:
    resultado = veb.existe(valor)
    print(f" -> existe({valor}): {resultado}")

  # 3. Teste de sucessores
  print("\n[3] Busca por sucessores:")
  for valor in [0, 5, 6, 9, 12]:
    resultado = veb.sucessor_de(valor)
    print(f" -> sucessor_de({valor}): {resultado}")

  # 4. Teste de antecessores
  print("\n[4] Busca por antecessores:")
  for valor in [2, 6, 9, 11, 13]:
    resultado = veb.antecessor_de(valor)
    print(f" -> antecessor_de({valor}): {resultado}")

  # 5. Visualização da estrutura por clusters
  print("\n[5] Estrutura interna (clusters):")
  clusters = veb.listar_por_cluster()
  for chave, itens in clusters.items():
    if chave == 'min':
      print(f" -> Mínimo atual: {itens}")
    else:
      print(f" -> Cluster {chave}: {itens}")

  # 6. Remoção de um elemento
  print("\n[6] Remoção do elemento 10:")
  veb.excluir(10)
  print(f" -> existe(10): {veb.existe(10)}")
  print(f" -> sucessor_de(8): {veb.sucessor_de(8)}")

  # 7. Visualização da estrutura após a remoção
  print("\n[7] Estrutura após remoção:")
  clusters = veb.listar_por_cluster()
  for chave, itens in clusters.items():
    if chave == 'min':
      print(f" -> Mínimo atual: {itens}")
    else:
      print(f" -> Cluster {chave}: {itens}")

# --------------------------------------------------
# 5. OPERAÇÕES POR MEIO DE UMA STRING ÚNICA
# --------------------------------------------------
def processar_com_string(texto_op: str):

  """
  Processa uma sequência de operações VEB (van Emde Boas) fornecida como string.

  Parâmetros:
    texto_op (str): Texto contendo operações linha por linha.

  Retorna:
    list: Lista de strings com os resultados das operações processadas.

  Operações válidas:
    - INC <val>: Insere o valor na estrutura.
    - REM <val>: Remove o valor da estrutura.
    - SUC <val>: Retorna o sucessor de <val>.
    - PRE <val>: Retorna o antecessor de <val>.
    - IMP: Imprime os clusters e o menor valor presente.
  """

  veb = EstruturaVEB(2**16)
  saidas = []

  for linha in texto_op.strip().split('\n'):
    partes = linha.strip().split()
    if not partes:
      continue
    op = partes[0]

    if op == "INC":
      val = int(partes[1])
      veb.adicionar(val)
      saidas.append(f"INC {val}")

    elif op == "REM":
      val = int(partes[1])
      veb.excluir(val)
      saidas.append(f"REM {val}")

    elif op == "SUC":
      val = int(partes[1])
      res = veb.sucessor_de(val)
      saidas.append(f"SUC {val} {res}" if res is not None else f"SUC {val}")

    elif op == "PRE":
      val = int(partes[1])
      res = veb.antecessor_de(val)
      saidas.append(f"PRE {val} {res}" if res is not None else f"PRE {val} None")

    elif op == "IMP":
      clusters = veb.listar_por_cluster()
      partes_out = ["IMP"] # Início da saída da impressão
      if 'min' in clusters:
        partes_out.append(f"Min: {clusters['min']}")
      for c in sorted(k for k in clusters if k != 'min'):
        lista = ", ".join(map(str, clusters[c]))
        partes_out.append(f"C[{c}]: {lista}")
      saidas.append(", ".join(partes_out))

  return saidas

# --------------------------------------------------
# 6. OPERAÇÕES A PARTIR DE UM ARQUIVO TXT
# --------------------------------------------------
def processar_txt(entrada: str, saida: str):
  veb = EstruturaVEB(2**32)
  with open(entrada, 'r') as fin, open(saida, 'w') as fout:
    for linha in fin:
      partes = linha.strip().split()
      if not partes:
        continue
      op = partes[0]

      if op == "INC":
        val = int(partes[1])
        veb.adicionar(val)
        fout.write(f"INC {val}\n")

      elif op == "REM":
        val = int(partes[1])
        veb.excluir(val)
        fout.write(f"REM {val}\n")

      elif op == "SUC":
        val = int(partes[1])
        res = veb.sucessor_de(val)
        fout.write(f"SUC {val}\n")
        if res is not None:
          fout.write(f"{res}\n")

      elif op == "PRE":
        val = int(partes[1])
        res = veb.antecessor_de(val)
        fout.write(f"PRE {val}\n")
        if res is not None:
          fout.write(f"{res}\n")
        else:
          fout.write("None\n")

      elif op == "IMP":
        fout.write("IMP\n")
        clusters = veb.listar_por_cluster()
        if 'min' in clusters:
          fout.write(f"Min: {clusters['min']}")
          del clusters['min']
        for k in sorted(clusters):
          vals = ", ".join(map(str, clusters[k]))
          fout.write(f", C[{k}]: {vals}")
        fout.write("\n")

# --------------------------------------------------
# 7. TESTE FUNCIONAL ALTERNATIVO SEM USO DE TXT
# --------------------------------------------------
def executar_teste_alternativo():
  print("\n\n=== Teste funcional alternativo da EstruturaVEB ===")
  operacoes_teste = """INC 10
  INC 2048
  INC 4096
  INC 32000
  INC 12345
  INC 876
  INC 432
  INC 1
  INC 65534
  IMP
  SUC 500
  PRE 500
  SUC 32000
  PRE 876
  REM 2048
  IMP
  SUC 1000
  PRE 900"""

  veb_32 = EstruturaVEB(2**32)
  resultados = []

  for linha in operacoes_teste.strip().split('\n'):
    partes = linha.strip().split()
    if not partes:
      continue
    op = partes[0]

    if op == "INC":
      val = int(partes[1])
      veb_32.adicionar(val)
      resultados.append(f"INC {val}")

    elif op == "REM":
      val = int(partes[1])
      veb_32.excluir(val)
      resultados.append(f"REM {val}")

    elif op == "SUC":
      val = int(partes[1])
      res = veb_32.sucessor_de(val)
      resultados.append(f"SUC {val} {res}" if res is not None else f"SUC {val}")

    elif op == "PRE":
      val = int(partes[1])
      res = veb_32.antecessor_de(val)
      resultados.append(f"PRE {val} {res}" if res is not None else f"PRE {val} None")

    elif op == "IMP":
      clusters = veb_32.listar_por_cluster()
      partes_out = ["IMP"]
      if 'min' in clusters:
        partes_out.append(f"Min: {clusters['min']}")
      for c in sorted(k for k in clusters if k != 'min'):
        lista = ", ".join(map(str, clusters[c]))
        partes_out.append(f"C[{c}]: {lista}")
      resultados.append(", ".join(partes_out))

  for r in resultados:
    print(r)

# --------------------------------------------------
# 8. MÉTODO PRINCIPAL DE EXECUÇÃO
# --------------------------------------------------
if __name__ == "__main__":

  testar_veb()

  print("\n\n=== Tentando processar entrada.txt (se existir) ===")
  try:
    processar_txt("entrada.txt", "saida.txt")
    print("Arquivo de saída gerado: saida.txt")
  except FileNotFoundError:
    print("Arquivo entrada.txt não encontrado. Ignorando esse passo.")

  executar_teste_alternativo()

=== Iniciando testes na estrutura EstruturaVEB ===

[1] Inserção de elementos: 1, 6, 8, 10, 12
 -> Inserido: 1
 -> Inserido: 6
 -> Inserido: 8
 -> Inserido: 10
 -> Inserido: 12

[2] Verificação de existência:
 -> existe(0): False
 -> existe(1): True
 -> existe(5): False
 -> existe(6): True
 -> existe(15): False

[3] Busca por sucessores:
 -> sucessor_de(0): 1
 -> sucessor_de(5): 6
 -> sucessor_de(6): 8
 -> sucessor_de(9): 10
 -> sucessor_de(12): None

[4] Busca por antecessores:
 -> antecessor_de(2): 1
 -> antecessor_de(6): 1
 -> antecessor_de(9): 8
 -> antecessor_de(11): 10
 -> antecessor_de(13): 12

[5] Estrutura interna (clusters):
 -> Mínimo atual: 1
 -> Cluster 1: [6]
 -> Cluster 2: [8, 10]
 -> Cluster 3: [12]

[6] Remoção do elemento 10:
 -> existe(10): False
 -> sucessor_de(8): 12

[7] Estrutura após remoção:
 -> Mínimo atual: 1
 -> Cluster 1: [6]
 -> Cluster 2: [8]
 -> Cluster 3: [12]


=== Tentando processar entrada.txt (se existir) ===
Arquivo de saída gerado: saida.txt


===

### Conclusão

Fazer esse trabalho foi um desafio, mas também uma ótima oportunidade de aprendizado. No começo, entender como funciona a estrutura van Emde Boas não foi nada fácil — a ideia de dividir o universo de chaves em partes menores e lidar com clusters de forma recursiva parecia confusa. Mas à medida que fui implementando cada parte com calma, tudo começou a fazer mais sentido.

Além da lógica principal da estrutura, também foi interessante entender como o uso de técnicas como table doubling e halving ajuda a manter o uso de memória eficiente, mesmo com um universo grande de valores possíveis.

No final, percebi que a vEB pode ser muito útil em alguns cenários específicos, mesmo que não seja tão conhecida quanto outras estruturas mais tradicionais.Ela é um ótimo exemplo de como a escolha da estrutura certa pode trazer ganhos expressivos de desempenho.